# **Compreenção e Prevenção aos Inadimplentes**

Os dados financeiros estão em formato CSV e contém informações sobre clientes de uma instuição financeira. Estamos interessados na coluna default, que indica se um cliente é adimplente(Está em dias com suas contas), ou inadimplente(Está com pendências financeiras). Ou seja, queremos entender o porque um cliente deixa de honrar com suas dívidas baseado no comportamento de outros atributos, como salário, escolaridade e movimentação financeira.

`  Default = 0 #Adimplente`
`  Default = 1 #Inadimplente`

# **Tópicos**
# 

1.   Exploração
2.   Transformação e Limpeza de dados
3.   Visualização de dados
4.   Storytelling


# 1. Exploração de dados financeiro

🔸- Vamos começar com biblioteca pandas

In [ ]:
import pandas as pd

🔸- Exportando o arquivo do banco de dados dentro do kaggle.

In [ ]:
df = pd.read_csv('/kaggle/input/dados01/dados.csv', sep=',', na_values='na')

🔸- Tabela dos dados

In [ ]:
df.head(n=10)

# **1.2 Estrutura**

🔸- Quantidade de Linhas e Colunas

In [ ]:
df.shape

🔸- Separação de dados da tabela

In [ ]:
qtd_total, _ = df.shape
qtd_adimplentes, _ = df[df['default'] == 0].shape
qtd_inadimplentes, _ = df[df['default'] == 1].shape

🔸- Divisão para determinar a proporção.

In [ ]:
print(f"A proporção de clientes adimplentes é de {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"A proporção de clientes inadimplentes é de {round(100 * qtd_inadimplentes / qtd_total, 2)}%")

# **1.3 Schema**

🔸- Verificando a tabela de dados novamente.

In [ ]:
df.head(n=5)

 🔸- Colunas e seus respectivos **tipos** de dados.

In [ ]:
df.dtypes

🔸- Atributos **categóricos**.

In [ ]:
df.select_dtypes('object').describe().transpose()

🔸- Atributos **númericos**.

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

# **1.4 - Dados faltantes**

🔸- Verificando a tabela de dados

In [ ]:
df.head()

🔸- Informa as categorias que tem dados faltando. [NaN]

In [ ]:
df.isna().any()

🔸- Imprime estatísticas sobre os dados faltantes, valores None

In [ ]:
def stats_dados_faltantes(df: pd.DataFrame) -> None:

  stats_dados_faltantes = []
  for col in df.columns:
    if df[col].isna().any():
      qtd, _ = df[df[col].isna()].shape
      total, _ = df.shape
      dict_dados_faltantes = {col: {'quantidade': qtd, "porcentagem": round(100 * qtd/total, 2)}}
      stats_dados_faltantes.append(dict_dados_faltantes)

  for stat in stats_dados_faltantes:
    print(stat)

In [ ]:
stats_dados_faltantes(df=df)

In [ ]:
stats_dados_faltantes(df=df[df['default'] == 0])

In [ ]:
stats_dados_faltantes(df=df[df['default'] == 1])

# 2.1 Transformação e limpeza

**Data Wrangling**

A seguinte vamos corrigir o schema das nossas colunas e remover os dados faltantes.

**Correção de schema**

Na etapa de exploração, notamos que as colunas **limite_credito** e **valor_transacoes_12m** estavam sendo interpretadas como colunas categóricas (`dtype = object`).

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].dtypes

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].head(n=5)

🔸- Criando uma função `lambda` para limpar os dados e usar o método funcional map.

In [ ]:
fn = lambda valor: float(valor.replace(".", "").replace(",", "."))

valores_originais = ['12.691,51', '8.256,96', '3.418,56', '3.313,03', '4.716,22']
valores_limpos = list(map(fn, valores_originais))

print(valores_originais)
print(valores_limpos)

🔸- Aplicando função `lambda` de limpeza nas colunas de interesse

In [ ]:
df['valor_transacoes_12m'] = df['valor_transacoes_12m'].apply(fn)
df['limite_credito'] = df['limite_credito'].apply(fn)

🔸- Verifica novamente se ainda está objeto, as colunas limite_credito, valor_transacoes_12m

In [ ]:
df.dtypes

🔸- Atributos **categóricos**

In [ ]:
df.select_dtypes('object').describe().transpose()

🔸- Atributos **númericos**.

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

# **3.2 - Remoção de dados faltantes**

Como o pandas está ciente do que é um dados faltante, a remoção das linhas problemáticas é trivial.

In [ ]:
df.dropna(inplace=True)

🔸- Vamos analisar novamente a estrutura dos dados

In [ ]:
df.shape

In [ ]:
df[df['default'] == 0].shape

In [ ]:
df[df['default'] == 1].shape

In [ ]:
qtd_total_novos, _ = df.shape
qtd_adimplentes_novos, _ = df[df['default'] == 0].shape
qtd_inadimplentes_novos, _ = df[df['default'] == 1].shape

In [ ]:
print(f"A proporção adimplentes ativos é de {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"A nova proporção de clientes adimplentes é de {round(100 * qtd_adimplentes_novos / qtd_total_novos, 2)}%")
print("")
print(f"A proporção de clientes inadimplentes é de {round(100 * qtd_inadimplentes / qtd_total, 2)}%")
print(f"A nova proporção de clientes inadimplentes é de {round(100 * qtd_inadimplentes_novos / qtd_total_novos, 2)}%")

# **4. Visualização de dados**

Vamos criar diversas visualizações para correlacionar variáveis explicativas com a variável resposta para buscar entender qual fator leva um cliente a inadimplencia. E para isso, vamos sempre comparar a base com todos os clientes com a base de adimplentes e inadimplentes.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings

sns.set_style("whitegrid")

In [ ]:
df_adimplente = df[df['default'] == 0]

In [ ]:
df_inadimplente = df[df['default'] == 1]

#**4.1 Visualização categóricas**

🔸- Vamos analisar relação entre variável resposta default com os atributos categóricos

In [ ]:
df.select_dtypes('object').head(n=5)

🔸- Visualização da coluna Escolaridade

In [ ]:
coluna = 'escolaridade'
titulos = ['Escolaridade dos Clientes', 'Escolaridade dos Clientes Adimplentes', 'Escolaridade dos Clientes Inadimplentes']

figura, eixos = plt.subplots(1, 3, figsize=(20, 5), sharex=True)
max_y = 0

for eixo, dataframe in enumerate([df, df_adimplente, df_inadimplente]):

    df_to_plot = dataframe[coluna].value_counts().reset_index()
    df_to_plot.columns = [coluna, 'frequencia_absoluta']
    df_to_plot.sort_values(by=[coluna], inplace=True)

    f = sns.barplot(data=df_to_plot, x=coluna, y='frequencia_absoluta', ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

    f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)

    _, max_y_f = f.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y

for eixo in eixos:
    eixo.set(ylim=(0, max_y))

plt.show()

🔸- Visualização do Salário Anual

In [ ]:
coluna = 'salario_anual'
titulos = ['Salario Anual dos Clientes', 'Salário Anual dos Clientes Adimplentes', 'Salário Anual dos Clientes Inadimplentes']

figura, eixos = plt.subplots(1, 3, figsize=(20, 5), sharex=True)
max_y = 0

for eixo, dataframe in enumerate([df, df_adimplente, df_inadimplente]):

    df_to_plot = dataframe[coluna].value_counts().reset_index()
    df_to_plot.columns = [coluna, 'frequencia_absoluta']
    df_to_plot.sort_values(by=[coluna], inplace=True)

    f = sns.barplot(data=df_to_plot, x=coluna, y='frequencia_absoluta', ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

    f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)

    _, max_y_f = f.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y

for eixo in eixos:
    eixo.set(ylim=(0, max_y))

plt.show()

*  Sálario menor que é 40k que é o mais baixo é adimplentes, enquanto salários de 120k são menores em adimplentes. Conclusão é que isso não interfere no comportamento inadimplente.  

🔸- Visualização da Idade

In [ ]:
coluna = 'estado_civil'
titulos = ['Estado Civil dos Clientes', 'Estado Civil dos Clientes Adimplentes', 'Estado Civil dos Clientes Inadimplentes']

figura, eixos = plt.subplots(1, 3, figsize=(20, 5), sharex=True)
max_y = 0

for eixo, dataframe in enumerate([df, df_adimplente, df_inadimplente]):

    df_to_plot = dataframe[coluna].value_counts().reset_index()
    df_to_plot.columns = [coluna, 'frequencia_absoluta']
    df_to_plot.sort_values(by=[coluna], inplace=True)

    f = sns.barplot(data=df_to_plot, x=coluna, y='frequencia_absoluta', ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

    f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)

    _, max_y_f = f.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y

for eixo in eixos:
    eixo.set(ylim=(0, max_y))

plt.show()

*  Lembrando que o gráfico está em porcentagem, no gráfico de inadimplência não há diferença os clientes que não pagam estão no mesmo gráfico entre casado e solteiro, então na minha conclusão Estado Civil não interfere no comportamento Inadimplente.

#**4.2. Visualizações numéricas**

In [ ]:
df.drop(['id', 'default'], axis=1).select_dtypes('number').head(n=5)

In [ ]:
coluna = 'valor_transacoes_12m'
titulos = ['Qtd. de Quansações no Último Ano', 'Qtd. de Transações no Último Ano de Adimplentes', 'Qtd. de Transações no Último Ano de Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20,3), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

  figura.show()



*   Podemos dizer que quanto mais transações menor a chance desse cliente se tornar inadimplente.


In [ ]:
warnings.filterwarnings("ignore")

f = sns.relplot(x='valor_transacoes_12m', y='qtd_transacoes_12m', data=df, hue='default')

_ = f.set(
    title='Relação entre valor e Quantidade de Transações no Último Ano',
    xlabel='Valor das Transações no Último Ano',
    ylabel='Quantidade das Transações no Último Ano'
)

plt.subplots_adjust(hspace=0.5, wspace=0.5)

plt.show()



*   Ficou claro que quanto mais transações menor o número do cliente ser inadimplente, podemos seguir esse padrão de pensamento para gerar uma solução.



🔸- Antes de avançarmos vamos verificar se o limite de crédito alto tende a formar mais inadimplentes

In [ ]:
coluna = 'limite_credito'
titulos = ['Limite de Crédito', 'Limite de Crédito de Adimplentes', 'Limite de Crédito de Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()



*   Nas extremidades no inadimplente entre valor de 5k - 35k há um número grande com limites altos, talvez um pouco de atenção abaixando o limite diminua um pouco a inadimplência, mas não influência a variável de interesse.



****
# **Conclusão**

1.   Nosso principal objetivo era entender e tentar prever o comportamente de clientes se tornarem inadimplentes com base nos dados fornecidos.
2.   Depois de diversos gráficos comparativos podemos concluir que os gráficos categóricos, como Salário, Escolaridade e Estado Civil, pouco diferem no comportamente de adimplente ou inadimplente.
3.   Já os gráficos comparativos de categória númericos tivemos mais sucesso em obter um resultado, com base nos gráfico conluimos que quanto menos transações e menor o valor transicionado, maior a chance de uma pessoa se tornar uma pessoa se tornar inadimplente.
4.   Vimos também que o limite de crédito possuem maiores inadimplentes, com baixo e alto número de crédito.

****

# **Conclusão Final**

Com as informações que possuimos podemos prever e tentar tratar clientes que possuam esse tipo de comportamente inadimplente, antes de liberar acréscimo de crédito.

****
